# LDPFL on the Medical MNIST Dataset

This notebook shows how to run LDPFL on the Medical MNIST Dataset.

#### Importing packages

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split, Subset
from torchvision import datasets
from torchvision.datasets import MNIST
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset
from sklearn.model_selection import train_test_split

import numpy as np
import math
import time
import os
import random
import statistics
import matplotlib.pyplot as plt
from scipy.io import loadmat
from Medical_MNIST_FL import FedMLFunc as fl
from Medical_MNIST_FL import Net as Net
from tqdm import tqdm
fl = fl()

from collections import OrderedDict
from collections import defaultdict
from typing import List, Tuple


from LDP_Functions import SRR
from LDP_Functions import LDP_FL
from LDP_Functions import GRR



# print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

In [ ]:
# Checking the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# Getting GPU usage information before computation
if device.type == 'cuda':
    for i in range(0,torch.cuda.device_count()):
        print(torch.cuda.get_device_name(i))
        print('Memory Usage of device :', i)
        print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [ ]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

#### Main Configuration

In [ ]:
np.random.seed(25)
num_classes = 10 

# Client training settings
localepochs = 12 # The number of epochs for local model training. 50 is the default value    
BATCH_SIZE = 64
weight_decay = 1e-4

# FL settings
num_of_clients = 100
num_selected = 10
num_rounds = 15 # 400 is the default value
epochs = 10 # The number of epochs for the clients during FL 
# batch_size = 32

#### Loading the data and preparing data for federated learning

In [ ]:

def load_datasets():
    
    train_transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
    
    train_data = torchvision.datasets.ImageFolder(root=("/home/rit/LDPFL-main/dataset/archive/"), transform = train_transform )
    
    train_indices, test_indices = train_test_split(list(range(len(train_data.targets))), test_size=0.1, stratify=train_data.targets)
    train_dataset = torch.utils.data.Subset(train_data, train_indices)
    test_dataset = torch.utils.data.Subset(train_data, test_indices)
    
    n_samples = len(train_dataset) // num_of_clients
    class_counts = torch.zeros(6)
    for i in range(len(train_dataset)):
        class_counts[train_dataset[i][1]] += 1
        
    
    print("Checkpoint 1")

    # Divide the samples for each class into n parts
    class_indices = {}
    for i in range(len(train_dataset)):
        label = train_dataset[i][1]
        if label not in class_indices:
            class_indices[label] = []
        class_indices[label].append(i)
        
    print("Checkpoint 2")

    for label in class_indices:
        np.random.shuffle(class_indices[label])
        class_indices[label] = [class_indices[label][i::num_of_clients] for i in range(num_of_clients)]
        
    print("Checkpoint 3")

    # Create datasets for each client by combining the parts from each class
    datasets_list = []
    for i in range(num_of_clients):
        indices = []
        for label in class_indices:
            indices += class_indices[label][i]
        dataset = torch.utils.data.Subset(train_dataset, indices)
        dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
        datasets_list.append(dataloader)
    
    test_loader=DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

    # Return train dataloaders and test dataloader
    return datasets_list, test_loader



trainloaders, testloader = load_datasets()

#### Model Summary

In [ ]:
from torchsummary import summary

net = Net()
net.to(device)
summary(net, (3, 224, 224))

#### Federated Learning

In [ ]:
SRR_PARAMS = [
#     [0.0, 0.075, 5, 0, 3, 10, 0.1],
    [0.0, 0.075, 5, 0, 4, 10, 0.1],
    [0.0, 0.075, 5, 0, 5, 10, 0.1],
    [0.0, 0.075, 5, 0, 6, 10, 0.1],
    [0.0, 0.075, 5, 0, 7, 10, 0.1],
    [0.0, 0.075, 5, -30, 4, 10, 0.1],
    [0.0, 0.075, 5, -20, 4, 10, 0.1],
    [0.0, 0.075, 5, -10, 4, 10, 0.1],
    [0.0, 0.075, 5, 0, 4, 10, 0.1],
    [0.0, 0.075, 5, 10, 4, 10, 0.1],
    [0.0, 0.075, 5, 20, 4, 10, 0.1],
    [0.0, 0.075, 5, 30, 4, 10, 0.1],
    [0.0, 0.075, 5, 20, 4, 10, 0.1],
    [0.0, 0.075, 5, 20, 4, 10, 0.2],
    [0.0, 0.075, 5, 20, 4, 10, 0.3],
    [0.0, 0.075, 5, 20, 4, 10, 0.4],
    [0.0, 0.075, 5, 20, 4, 10, 0.5],
    [0.0, 0.075, 5, 20, 4, 10, 0.6],
    [0.0, 0.075, 5, 20, 4, 10, 0.7],
    [0.0, 0.075, 5, 20, 4, 10, 0.8],
    [0.0, 0.075, 5, 20, 4, 10, 0.9],
    [0.0, 0.075, 5, 20, 4, 10, 0.95],
    [0.0, 0.075, 5, 20, 4, 10, 0.99],
    [0.0, 0.075, 5, 20, 4, 10, 1.0]
]

In [ ]:
for param in SRR_PARAMS:
    
    with open('medical_mnist_exp.txt', 'a+') as f:
        f.write('\n-------------------------------------------------------------\n')
        # f.write(f'\nSRR Parameters\nc = {param[0]}\nr = {param[1]}\ne = {param[2]}\nd = {param[3]}\np = {param[4]}\nm = {param[5]}\nf = {param[6]}\n\n')

    # print(f'\nSRR Parameters\nc = {param[0]}\nr = {param[1]}\ne = {param[2]}\nd = {param[3]}\np = {param[4]}\nm = {param[5]}\nf = {param[6]}\n\n')
    # Emptying CUDA cache
    torch.cuda.empty_cache()

    # Instantiate models and optimizers

    # Global Model
    global_model = nn.DataParallel(Net()).cuda()

    # Client Models as a list
    client_models = [nn.DataParallel(Net()) for _ in range(num_of_clients)]

    # Initializing client models with global model weights and then saving them as model_x.ckpt where x stands for it's ID
    for i, model in enumerate(client_models):
        model.load_state_dict(global_model.state_dict())
    #     torch.save(model.state_dict(), './models/model_{}.ckpt'.format(i+1))

    # Optimizers as a list
    opt = [optim.Adam(model.parameters(),lr=0.001) for model in client_models]


    if device.type == 'cuda':
        for i in range(0,torch.cuda.device_count()):
            print(torch.cuda.get_device_name(i))
            print('Memory Usage of device :', i)
            print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
            print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


    ldp_func = SRR(c = param[0], r = param[1], epsilon = param[2], delta_d = param[3] , precision = param[4], m = param[5])
    # ldp_func = GRR(c = 0.0, r = 0.075, epsilon = 3, precision = 4)


    acc_train_collect = []
    acc_test_collect = []
    loss_train_collect = []
    loss_test_collect = []

    index = 0

    for r in tqdm(range(num_rounds)):

        round_start = time.time()
        
        index += 1
        print(f"Round {index}\n")
        
        # select (num_of_clients - 1) clients randomly
        client_idx = np.random.permutation(num_of_clients)[:int(param[6]*num_of_clients)]
        
        trainloss = 0
        trainacc = 0
        loss = 0
        
        count = 0
        
        # clients update
        for i in client_idx:
            print(f"Client {i} training\n")
            count += 1
            
            
            client_models[i].cuda()
            opt_state = opt[i].state_dict()
            # new_opt = optim.SGD(client_models[i].parameters(), lr=0.004, momentum=0.9, weight_decay=5e-4)
            new_opt = optim.Adam(client_models[i].parameters(), lr=0.001)
            new_opt.load_state_dict(opt_state)
            
            # calling client_update
            [loss,acc]= fl.client_update(client_models[i], opt[i], trainloaders[i], epochs, device)  
            
            client_models[i].to('cpu')
            opt[i].load_state_dict(new_opt.state_dict())
            
            
            trainloss += loss
            trainacc += acc
            
            torch.cuda.empty_cache()
        
        # server aggregate
        
        start = time.time()
        # fl.server_aggregate_ldpfl(global_model, [client_models[i] for i in client_idx], client_models)
        # fl.server_aggregate(global_model, [client_models[i] for i in client_idx], client_models)
        # fl.server_aggregate_srr(global_model, [client_models[i] for i in client_idx], client_models, ldp_func)
        fl.server_aggregate(global_model, [client_models[i] for i in client_idx], client_models)
        # fl.server_aggregate_srr(global_model, [client_models[i] for i in client_idx], client_models, ldp_func)
        end = time.time()
        
        
        # Testing the global model
        test_loss, test_acc = fl.test(global_model, testloader, device)
        

        print('Avg Train loss %0.3g - Avg Train accuracy: %0.3g  - Test loss %0.3g - Test accuracy: %0.3f' % (trainloss / len(client_idx), trainacc / len(client_idx), test_loss, test_acc))
        
        round_end = time.time()
        print(f"Time taken for server aggregation : {end-start} seconds.")
        print(f"Time taken for Total round : {round_end-round_start} seconds.")
        
        with open('medical_mnist_exp.txt', 'a+') as f:
            f.write(f"\nRound {index}\n")
            f.write('Avg Train loss %0.3g - Avg Train accuracy: %0.3g  - Test loss %0.3g - Test accuracy: %0.3f' % (trainloss / len(client_idx), trainacc / len(client_idx), test_loss, test_acc))
            f.write(f"\nTime taken for server aggregation : {end-start} seconds.\n")
            f.write(f"Time taken for Total round : {round_end-round_start} seconds.\n") 

        acc_train_collect.append(trainacc / len(client_idx))
        acc_test_collect.append(test_acc)
        loss_train_collect.append(trainloss / len(client_idx))
        loss_test_collect.append(test_loss)
        print("---------------------------------------------------------\n")

    for model in client_models:
        del model
    for o in opt:
        del o
    del global_model

    if device.type == 'cuda':
        for i in range(0,torch.cuda.device_count()):
            print(torch.cuda.get_device_name(i))
            print('Memory Usage of device :', i)
            print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
            print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

    print("\n**********************************************************************\n")
  

print("Training and Evaluation completed!")


#### Plotting

In [ ]:
# %matplotlib inline

x1 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y1 = [58.5, 52.8, 92.2, 90.7, 94.7, 96.2, 94.4, 92.6, 93.9, 95.9]

x2 = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
y2 = [87.8, 95.2, 96.1, 96.4, 96.8, 95.5, 97.3, 97.7, 97.6, 97.9]

# plot the data
plt.plot(x1, y1, color='red', label='SRR (e=20)')
plt.plot(x2, y2, color='blue', label='LDP-FL (e=1)')

# set plot title and labels
plt.title('SRR vs LDP-FL for 100 clients (FedAvg Settings)')
plt.xlabel('Rounds')
plt.ylabel('Test Accuracy')

# set legend and show plot
plt.legend()
plt.show()